In [8]:
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import DataLoader, random_split
from gloss_dataset import GlossDataset
from gloss_model import GlossModel
from torch.utils.tensorboard import SummaryWriter


In [9]:
# get data for training
gd = GlossDataset()
input_size = gd[0][0].shape[1]
class_no = len(gd.classes)
input_size, class_no

(1596, 3)

In [10]:
# provide input and class size
model = GlossModel(input_size, class_no)
device = model.device
model.to(device)
optim = optim.Adam(model.parameters(), lr=1e-4)
loss_fn = nn.CrossEntropyLoss()


In [11]:
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(device=device))

NVIDIA GeForce GTX 1650


In [12]:
# initialize sumamry writer
writer=SummaryWriter()

In [13]:
# Create testing and training dataLoader from single dataset using random_split
# and also set training epoch
split_ratio = 0.8
batch_size = 1
train_size = int(split_ratio*len(gd))
test_size = len(gd)-train_size
train_data, test_data = random_split(gd, [train_size, test_size])
train_dl = DataLoader(train_data, batch_size=batch_size,
                      shuffle=True, pin_memory=False)
test_dl = DataLoader(test_data, batch_size=batch_size,
                     shuffle=True, pin_memory=False)
epoch = 1000


In [14]:
# Start model training
model.train()
for i in range(epoch+1):
    for x_train, y_train in train_dl:
        xtrain, y_train = x_train.to(device), y_train.to(device)
        out = model(x_train)
        loss = loss_fn(out, y_train)
        optim.zero_grad()
        loss.backward()
        optim.step()
    writer.add_scalar("Loss/epoch", loss.item(), i)
    if (i % 100 == 0):
        print(f"Loss/epoch : {loss.item(),i}")
    writer.close()


Loss/epoch : (3.1780612468719482, 0)


KeyboardInterrupt: 

In [ ]:
# Evaluate Test set
model.eval()
tot_loss = 0
tp = 0
tot_test = 0
with torch.no_grad():
    for x_test, y_test in test_dl:
        out = model(x_test)
        loss = loss_fn(out, y_train)
        tot_loss += loss
        y_pred = torch.argmax(torch.max(out, dim=0).values)
        y_test = torch.argmax(y_test)
        if y_pred == y_test:
            tp += 1
        tot_test += 1

# Much more data required for training and test set
acc = tp/tot_test
loss = tot_loss/tot_test
print(f"Test Acc {acc}\nTest Loss : {loss}\n")
print("Tot test Num : ", tot_test)
print("Tot Positive : ", tp)


In [1]:
# save the model
torch.save(model, "asl-recog_lstm.pt")


NameError: name 'torch' is not defined